In [ ]:
import os
import json
import re
import requests
from dotenv import load_dotenv
from flask import Flask, request, session, redirect, url_for, render_template
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import Tool
from langchain_groq import ChatGroq
from deep_translator import GoogleTranslator
from Bio import Entrez

load_dotenv()

app = Flask(__name__)
app.secret_key = os.getenv("FLASK_SECRET_KEY", "default_secret")
app.config["SESSION_PERMANENT"] = False
app.config["SESSION_TYPE"] = "filesystem"

class SerperSearchTool:
    def __init__(self, api_key):
        self.api_key = api_key

    def run(self, query):
        url = "https://google.serper.dev/search"
        headers = {"X-API-KEY": self.api_key, "Content-Type": "application/json"}
        data = {"q": query}
        try:
            resp = requests.post(url, json=data, headers=headers)
            resp.raise_for_status()
            results = resp.json().get("organic", [])
            return "\n\n".join([f"{r['title']}\n{r['link']}\n{r['snippet']}" for r in results[:3]])
        except Exception as e:
            return f"حدث خطأ أثناء البحث: {str(e)}"

serper_tool = SerperSearchTool(api_key=os.getenv("SERPER_API_KEY"))


class PubMedSearch:
    def __init__(self, email=os.getenv("PUBMED_EMAIL")):
        Entrez.email = email

    def run(self, query):
        try:
            handle = Entrez.esearch(db="pubmed", term=query, retmax=5)
            record = Entrez.read(handle)
            ids = record["IdList"]
            if not ids:
                return "لا توجد نتائج في PubMed."
            summaries = Entrez.esummary(db="pubmed", id=",".join(ids))
            results = Entrez.read(summaries)
            return "\n\n".join([f"{item['Title']} ({item['PubDate']})" for item in results])
        except Exception as e:
            return f"حدث خطأ أثناء البحث في PubMed: {str(e)}"

pubmed_tool = Tool(
    name="بحث PubMed",
    func=PubMedSearch().run,
    description="ابحث في قاعدة بيانات PubMed للحصول على مقالات طبية موثوقة."
)

class TrustedMedicalSearch:
    def __init__(self, search_func):
        self.search = search_func

    def run(self, query):
        sources = ["site:who.int", "site:nhs.uk","site:nih.gov", "site:mayoclinic.org", "site:labtestsonline.org"]

        out = []
        for src in sources:
            out.append(self.search.run(f"{query} {src}"))
        return "\n\n".join(out)

trusted_tool = Tool(
    name="بحث طبي موثوق",
    func=TrustedMedicalSearch(serper_tool).run,
    description="ابحث فقط في Mayo Clinic، NHS، who ، Lab Tests Online."
)


def clean_question(text):
    text = text.lower()
    text = re.sub(r"[^\w\s]", "", text)
    return text.strip()

def safe_translate(text, target="ar", chunk_size=4500):
    translator = GoogleTranslator(source="auto", target=target)
    if len(text) <= chunk_size:
        return translator.translate(text)
    parts = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]
    return "\n".join(translator.translate(part) for part in parts)

class AISystem:
    def __init__(self):
        self.llm = ChatGroq(
            temperature=0,
            model_name="llama3-70b-8192",
            max_tokens=4096,
            groq_api_key=os.getenv("GROQ_API_KEY")
        )

        self.prompt_ar = ChatPromptTemplate.from_messages([
            ("system", 
            "أنت مساعد طبي ذكي وموثوق. أجب عن الأسئلة الطبية باستخدام اللغة العربية فقط، ولا تستخدم اللغة الإنجليزية في الرد إطلاقًا. "
            "أجب بطريقة واضحة، علمية، ومنظمة. استخدم النقاط والعناوين الفرعية والتنسيق المناسب عند الحاجة. "
            "ابدأ دائمًا بملخص مختصر للموضوع، ثم قدم التفاصيل بشكل منظم. "
            "اعتمد فقط على معلومات من مصادر طبية موثوقة مثل: منظمة الصحة العالمية، هيئة الخدمات الصحية الوطنية البريطانية، "
            " والمعهد الوطني للصحة الأمريكي و مايو كلينك، وموقع لاب تيستس أونلاين"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
            ("human", "{input}")
        ])


        self.prompt_en = ChatPromptTemplate.from_messages([
            ("system", 
            "You are a smart and reliable medical assistant. Answer medical questions in a clear, scientific, and structured way. Use bullet points and proper formatting when appropriate. Start with a short summary of the topic, then provide details in an organized form. Always rely on information from trusted sources like PubMed,nih,who ,Mayo Clinic, NHS, or LabTestsOnline."),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
            ("human", "{input}")
        ])

        tools = [trusted_tool , pubmed_tool]
        self.agent_ar = create_tool_calling_agent(llm=self.llm, tools=tools, prompt=self.prompt_ar)
        self.agent_en = create_tool_calling_agent(llm=self.llm, tools=tools, prompt=self.prompt_en)
        self.executor_ar = AgentExecutor(agent=self.agent_ar, tools=tools, verbose=True)
        self.executor_en = AgentExecutor(agent=self.agent_en, tools=tools, verbose=True)

        self.cache_path = "cache.json"
        self.cache = json.load(open(self.cache_path, "r", encoding="utf-8")) if os.path.exists(self.cache_path) else {}
        self.questions_ar = json.load(open("questions_ar.json", "r", encoding="utf-8")) if os.path.exists("questions_ar.json") else {}
        self.sperm_keywords = [
            'sperm', 'الحيوانات المنويه', 'الحيوانات المنوية', 'حيوان منوي',
            'سائل منوي', 'خصوبة', 'تحليل المني', 'تشوهات الحيوانات المنوية'
        ]

    def save_cache(self):
        json.dump(self.cache, open(self.cache_path, "w", encoding="utf-8"), ensure_ascii=False, indent=2)

    def get_sperm_info(self, question: str) -> str:
        q = clean_question(question)

        for k, v in self.questions_ar.items():
            if clean_question(k) in q:
                return v


        if q in self.cache:
            return self.cache[q]

        try:
            is_arabic = any('\u0600' <= c <= '\u06FF' for c in question)
            executor = self.executor_ar if is_arabic else self.executor_en
            result = executor.invoke({"input": question})["output"]

    
            if is_arabic and not all('\u0600' <= c <= '\u06FF' for c in result):
                result = safe_translate(result, target="ar")

            self.cache[q] = result
            self.save_cache()
            return result

        except Exception as e:
            return f"حدث خطأ أثناء جلب المعلومات: {safe_translate(str(e), target='ar')}"



    def get_response(self, question: str) -> str:
        is_ar = any('\u0600' <= c <= '\u06FF' for c in question)
        q_clean = clean_question(question)


        if any(kw in q_clean for kw in self.sperm_keywords):
            return self.get_sperm_info(question)


        if is_ar and q_clean in self.questions_ar:
            return self.questions_ar[q_clean]

       
        if q_clean in self.cache:
            return self.cache[q_clean]


        try:
            executor = self.executor_ar if is_ar else self.executor_en
            result = executor.invoke({"input": question})["output"]

            if is_ar and not all('\u0600' <= c <= '\u06FF' for c in result):
                result = safe_translate(result, target="ar")

            self.cache[q_clean] = result
            self.save_cache()
            return result

        except Exception as e:
            return f"حدث خطأ أثناء المعالجة: {safe_translate(str(e), target='ar')}"


   
ai_system = AISystem()

@app.route("/", methods=["GET", "POST"])
def index():
    session.setdefault("history", [])
    if request.method == "POST":
        q = request.form.get("question", "").strip()
        if q:
            session["history"].append(("user", q))
            ans = ai_system.get_response(q)
            session["history"].append(("bot", ans))
            session["history"] = session["history"][-4:]
            session.modified = True
        return redirect(url_for("index"))
    return render_template("index.html", history=session["history"])

if __name__ == "__main__":
    app.run(debug=False)
